## 事前準備

1. Google Cloud の新しいプロジェクトを作成します。
2. Cloud Shell を開いて、次のコマンドで API を有効化します。
```
gcloud services enable \
  aiplatform.googleapis.com \
  notebooks.googleapis.com \
  cloudresourcemanager.googleapis.com

```
3. 次のコマンドで Vertex AI Workbench のインスタンスを作成します。
```
PROJECT_ID=$(gcloud config list --format 'value(core.project)')
gcloud workbench instances create agent-development \
  --project=$PROJECT_ID \
  --location=us-central1-a \
  --machine-type=e2-standard-2
```

4. クラウドコンソールのナビゲーションメニューから「Vertex AI」→「Workbench」を選択すると、作成したインスタンス agent-develpment があります。インスタンスの起動が完了するのを待って、「JUPYTERLAB を開く」をクリックしたら、「Python 3(ipykernel)」の新規ノートブックを作成します。

5. この後は、ノートブック上で以下のコマンドを実行していきます。

## ADK パッケージのインストール

In [ ]:
%pip install --upgrade --user google-adk

インストールしたパッケージを利用可能にするために、次のコマンドでカーネルを再起動します。

再起動を確認するポップアップが表示されるので [Ok] をクリックします。

In [2]:
import IPython
app = IPython.Application.instance()
_ = app.kernel.do_shutdown(True)

## Search Agent App の作成

In [1]:
import copy, datetime, json, os, pprint, time, uuid
import vertexai
from google.genai.types import Part, UserContent, ModelContent
from google.adk.agents.llm_agent import LlmAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools import google_search

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
LOCATION = 'us-central1'

vertexai.init(project=PROJECT_ID, location=LOCATION,
              staging_bucket=f'gs://{PROJECT_ID}')

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

In [53]:
instruction = '''
You are a friendly AI assistant that answers user's queries.
Use google_search to give answers based on the latest and objective information.

[Format instruction]
Output in Japanese, in plain text only without any special formatting characters.
Avoid adding citation marks such as [1][2].
'''

search_agent = LlmAgent(
    name='search_agent',
    model='gemini-2.0-flash-001',
    description='Agent to answer questions using Google Search.',
    instruction=instruction,
    tools=[google_search]
)

In [54]:
class LocalApp:
    def __init__(self, agent, app_name, user_id):
        self._agent = agent
        self._app_name = app_name
        self._user_id = user_id
        self._runner = Runner(
            app_name=self._app_name,
            agent=self._agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )
        self._session = self._runner.session_service.create_session(
            app_name=self._app_name,
            user_id=self._user_id,
            state={},
            session_id=uuid.uuid4().hex,
        )
        
    async def stream(self, query):
        content = UserContent(parts=[Part.from_text(text=query)])
        async_events = self._runner.run_async(
            user_id=self._user_id,
            session_id=self._session.id,
            new_message=content,
        )
        result = []
        async for event in async_events:
            if (event.content and event.content.parts):
                response = '\n'.join([p.text for p in event.content.parts if p.text])
                if response:
                    print(response)
                    result.append(response)
        return result

## 実行例

In [66]:
client = LocalApp(search_agent, 'Search Agent App', 'user00')

query = '''
高田馬場のおすすめのカレー屋は？
'''
_ = await client.stream(query)

高田馬場でおすすめのカレー屋さんについて調べてみます。


 高田馬場には美味しいカレー屋さんがたくさんありますね！おすすめのお店をいくつかご紹介します。

*   **カレーライス専門店ブラザー:** 鯖キーマカレーが人気の有名店です。スパイスの効いたカレーで、他では味わえない独特の味が楽しめます。
*   **横浜ボンベイ 高田馬場店:** こちらも人気のカレー店で、特にカシミールカレーがおすすめです。辛口ですが、野菜の甘みとスパイスの爽快感が絶妙にマッチしています。
*   **プネウマカレー:** コスパの良いカレー屋さんとして知られています。大盛りでもリーズナブルな価格で楽しめます。
*   **エチオピアカリーキッチン 高田馬場店:** こちらはトロトロのカレーが特徴で、長時間煮込んだ野菜の旨味がたっぷりです。
*   **白カレーの店 1/f ゆらぎ:** 珍しい白カレーが味わえるお店です。

これらの他にも、高田馬場にはたくさんのカレー屋さんがあります。ぜひ色々試して、お好みのカレーを見つけてみてください。



SessionService が管理するセッション情報を確認します。

セッションを特定するのに必要な、`app_name`、`user_id`、`session_id` を確認します。

In [67]:
app_name = client._session.app_name
user_id = client._session.user_id
session_id = client._session.id

app_name, user_id, session_id

('Search Agent App', 'user00', '1ee13edf707d441ea366e20b33c2f599')

確認した情報を使って、セッションに保存された情報を確認します。

2 つのイベント（ユーザーの質問とエージェントの回答）が記録されています。

エージェントの回答を生成する際に用いた Google Serch の情報なども確認できます。

In [68]:
session = client._runner.session_service.get_session(
    app_name = app_name,
    user_id = user_id,
    session_id = session_id,
)

session

Session(id='1ee13edf707d441ea366e20b33c2f599', app_name='Search Agent App', user_id='user00', state={}, events=[Event(content=UserContent(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='\n高田馬場のおすすめのカレー屋は？\n')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-f1b46067-8d00-4828-8279-cea7443b9c58', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='HxMzRwuI', timestamp=1747563352.992289), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text

次のように、これまでのやり取りが「イベント」として記録されています。

In [69]:
def format_timestamp(ts):
    return datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S.%f')

for event in session.events:
    print(format_timestamp(event.timestamp), event.author)

2025-05-18 10:15:52.992289 user
2025-05-18 10:15:52.992859 search_agent


特に、下記の要素から Google 検索に使ったキーワードが確認できます。

In [70]:
from IPython.display import display, HTML

search_queries = session.events[-1].grounding_metadata.web_search_queries
entry_point_html = session.events[-1].grounding_metadata.search_entry_point.rendered_content

print(search_queries)
display(HTML(entry_point_html))

['高田馬場 おすすめ カレー', '高田馬場 カレー 有名店', '高田馬場 カレー 人気ランキング']


続けて質問すると、セッション情報を踏まえた回答が得られます。

In [71]:
query = '''
特に家族連れにおすすめなのは？
'''
_ = await client.stream(query)

家族連れにおすすめの高田馬場のカレー屋さんについて調べてみます。


 家族連れにおすすめとなると、以下の点が重要になるかと思います。

*   **子供向けメニューやサービス**: 子供が食べやすいメニューがあるか、子供用の椅子や食器があるかなど。
*   **広々とした空間**: ベビーカーを置けるスペースがあるか、子供が多少騒いでも大丈夫な雰囲気か。
*   **アレルギー対応**: 食物アレルギーがある場合、対応してもらえるか。

これらの点を考慮すると、以下のカレー屋さんが候補として挙げられます。

*   **プネウマカレー:** mamacoというサイトで、子連れママ向けの情報が掲載されています。店内はカウンター席のみですが、空間が広々としており、ベビーカーを置くことができます。離乳食の持ち込みも可能です。
*   **エチオピアカリーキッチン 高田馬場店:** トロトロのカレーは子供にも食べやすいかもしれません。
*   **CoCo壱番屋:** メニューが豊富で、辛さも調整できるため、子供から大人まで楽しめます。子供向けのメニューやお子様カレーなどがある場合があります。

食べログなどのグルメサイトでは、子連れで利用しやすいカレー屋さんの情報や口コミが掲載されているので、参考にしてみてください。お店に直接電話して、子供向けメニューやサービス、アレルギー対応について確認することもおすすめです。



先ほどと同様にセッションに記録されたイベントを確認すると、新しい質問と回答のイベントが追加されています。

In [73]:
for event in session.events:
    print(format_timestamp(event.timestamp), event.author)

2025-05-18 10:15:52.992289 user
2025-05-18 10:15:52.992859 search_agent
2025-05-18 10:16:05.487202 user
2025-05-18 10:16:05.487850 search_agent


In [74]:
search_queries = session.events[-1].grounding_metadata.web_search_queries
entry_point_html = session.events[-1].grounding_metadata.search_entry_point.rendered_content

print(search_queries)
display(HTML(entry_point_html))

['高田馬場 おすすめ レストラン 子供', '高田馬場 カレー 子連れ', '高田馬場 カレー 家族連れ', '高田馬場 レストラン 家族', '高田馬場 カレー 子供']


## システムインストラクションの確認

ちょっとした裏技を使って、LlmAgent オブジェクトが Gemini API に送っている生の情報を覗いてみます。

In [75]:
import pprint
from google.adk.agents.invocation_context import InvocationContext
from typing import AsyncGenerator
from google.adk.events.event import Event
from google.adk.models.llm_request import LlmRequest

async def _run_one_step_async(
      self,
      invocation_context: InvocationContext,
  ) -> AsyncGenerator[Event, None]:
    """One step means one LLM call."""
    llm_request = LlmRequest()

    # Preprocess before calling the LLM.
    async for event in self._preprocess_async(invocation_context, llm_request):
        yield event
    if invocation_context.end_invocation:
        return

    # Calls the LLM.
    model_response_event = Event(
        id=Event.new_id(),
        invocation_id=invocation_context.invocation_id,
        author=invocation_context.agent.name,
        branch=invocation_context.branch,
    )

    ## DEBUG output
    if DEBUG:
        print('## Prompt contents ##')
        pprint.pp(llm_request.contents)
        print('----')
        print('## System instruction ##')
        print(llm_request.config.system_instruction)
        print('----')
        print('## Tools config ##')
        pprint.pp(llm_request.config.tools)
        print('----')
    ####

    async for llm_response in self._call_llm_async(
        invocation_context, llm_request, model_response_event
    ):
        # Postprocess after calling the LLM.
        async for event in self._postprocess_async(
            invocation_context, llm_request, llm_response, model_response_event
        ):
            yield event

from google.adk.flows.llm_flows.base_llm_flow import BaseLlmFlow
_run_one_step_async_original = copy.deepcopy(BaseLlmFlow._run_one_step_async)
BaseLlmFlow._run_one_step_async = _run_one_step_async

In [76]:
client = LocalApp(search_agent, 'Search Agent App', 'user00')

DEBUG = True
query = '''
高田馬場のおすすめのカレー屋は？
'''
_ = await client.stream(query)

## Prompt contents ##
[UserContent(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='\n高田馬場のおすすめのカレー屋は？\n')], role='user')]
----
## System instruction ##

You are a friendly AI assistant that answers user's queries.
Use google_search to give answers based on the latest and objective information.

[Format instruction]
Output in Japanese, in plain text only without any special formatting characters.
Avoid adding citation marks such as [1][2].


You are an agent. Your internal name is "search_agent".

 The description about you is "Agent to answer questions using Google Search."
----
## Tools config ##
[Tool(retrieval=None, google_search=GoogleSearch(), google_search_retrieval=None, enterprise_web_search=None, google_maps=None, code_execution=None, function_declarations=None)]
----
高田馬場のおすすめのカレー屋について調べてみます。

高田馬場にはたくさんのカレー屋さんがありますね。おすすめのお店をいくつかご紹介します。

*   **カ

In [77]:
query = '''
特に家族連れにおすすめなのは？
'''
_ = await client.stream(query)

## Prompt contents ##
[UserContent(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='\n高田馬場のおすすめのカレー屋は？\n')], role='user'),
 Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='高田馬場のおすすめのカレー屋について調べてみます。\n'), Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='高田馬場にはたくさんのカレー屋さんがありますね。おすすめのお店をいくつかご紹介します。\n\n*   **カレーライス専門店ブラザー:** 鯖キーマカレーが人気のお店です。\n*   **横浜ボンベイ 高田馬場店:** サラッとしたルーが特徴で、カシミールカレーがおすすめです。\n*   **プネウマカレー:** コスパの良いチキンカレーが人気です。\n*   **エチオピアカリーキッチン 高田馬場店:** 長時間煮込んだ野菜が美味しいと評判です。\n*    **白カレーの店 1/f ゆらぎ:** 珍しい白カレーが味わえます。\n\nこの他にも、Rettyや食べログなどのグルメサイトで、高田馬場のカレー店がたくさん紹介されています。ぜひ色々調べて、お好みのカレー屋さんを見つけてみてください

`## Prompt contents ##` の部分を見ると、セッションに記録されたイベントをすべてプロンプトに含めるのではなく、コンテキスト（これまでの会話の流れ）を把握するのに必要な情報が選択されていることがわかります。

裏技で変更した内部関数を元に戻しておきます。

In [78]:
DEBUG = False
BaseLlmFlow._run_one_step_async = _run_one_step_async_original

## AgentEngine へのデプロイ

In [79]:
from vertexai import agent_engines

remote_agent = agent_engines.create(
    agent_engine=search_agent,
    requirements=[
        'google-cloud-aiplatform[adk,agent_engines]',
    ]
)

Deploying google.adk.agents.Agent as an application.
Identified the following requirements: {'google-cloud-aiplatform': '1.88.0', 'cloudpickle': '3.1.1'}
The following requirements are missing: {'cloudpickle'}
The following requirements are appended: {'cloudpickle==3.1.1'}
The final list of requirements: ['google-cloud-aiplatform[adk,agent_engines]', 'cloudpickle==3.1.1']
Using bucket etsuji-15pro-poc
Wrote to gs://etsuji-15pro-poc/agent_engine/agent_engine.pkl
Writing to gs://etsuji-15pro-poc/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://etsuji-15pro-poc/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/879055303739/locations/us-central1/reasoningEngines/2482547721936830464/operations/7072978359533174784
View progress and logs at https://console.cloud.google.com/logs/query?project=etsuji-15pro-poc
AgentEngine created. Resource name: projects/879055303739/locations/us-central1/reasoningEngines/248

In [80]:
class RemoteApp:
    def __init__(self, remote_agent, user_id):
        self._remote_agent = remote_agent
        self._user_id = user_id
        self._session = remote_agent.create_session(user_id=self._user_id)
    
    def _stream(self, query):
        events = self._remote_agent.stream_query(
            user_id=self._user_id,
            session_id=self._session['id'],
            message=query,
        )
        result = []
        for event in events:
            if ('content' in event and 'parts' in event['content']):
                response = '\n'.join(
                    [p['text'] for p in event['content']['parts'] if 'text' in p]
                )
                if response:
                    print(response)
                    result.append(response)
        return result

    def stream(self, query):
        # Retry 4 times in case of resource exhaustion 
        for c in range(4):
            if c > 0:
                time.sleep(2**(c-1))
            result = self._stream(query)
            if result:
                return result
            if DEBUG:
                print('----\nRetrying...\n----')
        return None # Permanent error

In [81]:
remote_client = RemoteApp(remote_agent, 'user00')

query = '''
今年のゴールデンウィークは、カレンダー通りで何連休でしょうか？
'''
_ = remote_client.stream(query)

今年のゴールデンウィークがカレンダー通りだとすると何連休になるか調べてみます。


 2025年のゴールデンウィークは、カレンダー通りの場合、5月3日（土）から5月6日（火・振替休日）までの4連休となります。ただし、4月30日、5月1日、2日を休むことができれば、4月29日から5月6日まで最大8連休にすることも可能です。



In [82]:
query = '''
来年はどうなりますか？
'''
_ = remote_client.stream(query)

来年のゴールデンウィークがカレンダー通りだとすると何連休になるか調べてみます。

 2026年のゴールデンウィークは、カレンダー通りだと5月2日（土）から5月6日（水）までの5連休です。4月29日（水）は昭和の日ですが、飛び石の休みになります。



セッション情報を参照する際は、`VertexAiSessionService` を使用します。

In [83]:
app_name = remote_client._session['app_name']
user_id = remote_client._session['user_id']
session_id = remote_client._session['id']

app_name, user_id, session_id

('2482547721936830464', 'user00', '938345763132407808')

In [84]:
from google.adk.sessions import VertexAiSessionService

session_service = VertexAiSessionService(
    project = PROJECT_ID,
    location = LOCATION,
)

session = session_service.get_session(
    app_name=app_name,
    user_id=user_id,
    session_id=session_id,
)

session

Session(id='938345763132407808', app_name='2482547721936830464', user_id='user00', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='\n今年のゴールデンウィークは、カレンダー通りで何連休でしょうか？\n')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-bb52e2e9-fa00-4557-8530-208b5f801747', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='1372362585034194944', timestamp=1747563659.243566), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response

In [85]:
for event in session.events:
    print(format_timestamp(event.timestamp), event.author)

2025-05-18 10:20:59.243566 user
2025-05-18 10:20:59.306324 search_agent
2025-05-18 10:21:01.911222 user
2025-05-18 10:21:01.987574 search_agent


## 後片付け

デプロイ済みのエージェントをまとめて削除します。

In [ ]:
for agent in agent_engines.list():
    print(agent.gca_resource.name)
    agent.delete(force=True)